1. **Prepare o Dataset**:
   - Crie o arquivo `sales_data.csv` na pasta `data/input/` com o seguinte conteúdo:
     ```csv
     id,product,category,region,sales_amount,transaction_date
     1,Product A,Category 1,North,100,2023-01-01
     2,Product B,Category 1,North,200,2023-01-02
     3,Product C,Category 2,South,300,2023-01-03
     4,Product D,Category 2,South,400,2023-01-04
     5,Product E,Category 1,North,150,2023-01-05
     ```

In [1]:
import pandas as pd

# Dados fictícios
data = {
    "id": [1, 2, 3, 4, 5],
    "product": ["Product A", "Product B", "Product C", "Product D", "Product E"],
    "category": ["Category 1", "Category 1", "Category 2", "Category 2", "Category 1"],
    "region": ["North", "North", "South", "South", "North"],
    "sales_amount": [100, 200, 300, 400, 150],
    "transaction_date": ["2023-01-01", "2023-01-02", "2023-01-03", "2023-01-04", "2023-01-05"]
}

In [2]:
# Criar DataFrame
df = pd.DataFrame(data)
print(df)

   id    product    category region  sales_amount transaction_date
0   1  Product A  Category 1  North           100       2023-01-01
1   2  Product B  Category 1  North           200       2023-01-02
2   3  Product C  Category 2  South           300       2023-01-03
3   4  Product D  Category 2  South           400       2023-01-04
4   5  Product E  Category 1  North           150       2023-01-05


In [3]:
import os

# Caminho para salvar o arquivo
output_path = "../data/input/sales_data.csv"

# Garantir que o diretório existe
os.makedirs(os.path.dirname(output_path), exist_ok=True)

In [4]:
# Salvar como CSV
df.to_csv(output_path, index=False)

print(f"Dataset criado em {output_path}")

Dataset criado em ../data/input/sales_data.csv


2. **Execute as Seções**:
   - Execute cada célula no Jupyter Notebook.
   - Adapte as queries para diferentes cenários.

In [5]:
import pandas as pd

# Carregar o dataset
data_path = "../data/input/sales_data.csv"
df = pd.read_csv(data_path)

# Visualizar as primeiras linhas
print(df.head())


   id    product    category region  sales_amount transaction_date
0   1  Product A  Category 1  North           100       2023-01-01
1   2  Product B  Category 1  North           200       2023-01-02
2   3  Product C  Category 2  South           300       2023-01-03
3   4  Product D  Category 2  South           400       2023-01-04
4   5  Product E  Category 1  North           150       2023-01-05


## Configurar SQLite no Notebook ##

**Instalar SQLite e bibliotecas necessárias (se ainda não estiverem instaladas):**


In [6]:
import sqlite3
import pandas as pd

In [7]:
# Conectar ao banco SQLite (cria o arquivo "example.db" no diretório atual)
conn = sqlite3.connect("example.db")

In [8]:
# Criar uma tabela no banco de dados usando o DataFrame existente
df.to_sql("sales", conn, if_exists="replace", index=False)

print("Banco de dados configurado com sucesso!")

Banco de dados configurado com sucesso!


### Consultas Básicas ###

In [12]:
# Projeções e Filtros
query = """
SELECT
    region,
    product,
    sales_amount
FROM
    sales
WHERE
    region = 'North'
ORDER BY
    sales_amount DESC
"""

result = pd.read_sql_query(query, conn)
print("Projeções e Filtros:\n", result)


Projeções e Filtros:
   region    product  sales_amount
0  North  Product B           200
1  North  Product E           150
2  North  Product A           100


In [14]:
# Ordenação: Ordenar as vendas em ordem decrescente
query = """
SELECT
    product,
    sales_amount
FROM
    sales
ORDER BY
    sales_amount DESC
"""
result = pd.read_sql_query(query, conn)
print(result)


     product  sales_amount
0  Product D           400
1  Product C           300
2  Product B           200
3  Product E           150
4  Product A           100


In [15]:
# Agregações: Soma e Agrupamento

query = """
SELECT 
    region, 
    SUM(sales_amount) AS total_sales 
FROM sales 
GROUP BY region
"""

result = pd.read_sql_query(query, conn)
print("\nSoma de vendas por região:\n", result)


Soma de vendas por região:
   region  total_sales
0  North          450
1  South          700


In [19]:
# Agregação com Having: Filtrar regiões com total de vendas maior que 300

query = """
SELECT
    region,
    SUM(sales_amount) AS total_sales
FROM
    sales
GROUP BY
    region
HAVING
    total_sales >= 300
"""
result = pd.read_sql_query(query, conn)
print(result)


  region  total_sales
0  North          450
1  South          700


### Funções de Janela (Windows Functions) ###

**ROW_NUMBER**

In [20]:
# Atribuir um número de linha para cada produto dentro de uma região

query = """
SELECT
    region,
    product,
    sales_amount,
    ROW_NUMBER() OVER (PARTITION BY region ORDER BY sales_amount DESC) AS row_num
FROM
    sales
"""
result = pd.read_sql_query(query, conn)
print(result)


  region    product  sales_amount  row_num
0  North  Product B           200        1
1  North  Product E           150        2
2  North  Product A           100        3
3  South  Product D           400        1
4  South  Product C           300        2


**RANK**

In [21]:
# Ranqueamento de vendas por região

query = """
SELECT
    region,
    product,
    sales_amount,
    RANK() OVER (PARTITION BY region ORDER BY sales_amount DESC) AS rank
FROM
    sales
"""
result = pd.read_sql_query(query, conn)
print(result)


  region    product  sales_amount  rank
0  North  Product B           200     1
1  North  Product E           150     2
2  North  Product A           100     3
3  South  Product D           400     1
4  South  Product C           300     2


**PARTITION BY com soma acumulada**

In [22]:
# Soma acumulada das vendas por região

query = """
SELECT
    region,
    product,
    sales_amount,
    SUM(sales_amount) OVER (PARTITION BY region ORDER BY transaction_date) AS running_total
FROM
    sales
"""
result = pd.read_sql_query(query, conn)
print(result)


  region    product  sales_amount  running_total
0  North  Product A           100            100
1  North  Product B           200            300
2  North  Product E           150            450
3  South  Product C           300            300
4  South  Product D           400            700


### Exploração Avançada ###

**Percentual de Vendas por Região**

In [24]:
# Calcular o percentual de vendas por região
query = """
SELECT
    region,
    SUM(sales_amount) AS total_sales,
    ROUND(SUM(sales_amount) * 100.0 / (SELECT SUM(sales_amount) FROM sales), 2) AS sales_percentage
FROM
    sales
GROUP BY
    region
"""
result = pd.read_sql_query(query, conn)
print(result)


  region  total_sales  sales_percentage
0  North          450             39.13
1  South          700             60.87


**Top N Produtos em Vendas por Região**

In [25]:
# Selecionar os 2 produtos mais vendidos em cada região
query = """
SELECT
    region,
    product,
    sales_amount
FROM (
    SELECT
        region,
        product,
        sales_amount,
        RANK() OVER (PARTITION BY region ORDER BY sales_amount DESC) AS rank
    FROM
        sales
)
WHERE
    rank <= 2
"""
result = pd.read_sql_query(query, conn)
print(result)


  region    product  sales_amount
0  North  Product B           200
1  North  Product E           150
2  South  Product D           400
3  South  Product C           300


In [26]:
conn.close()